<a href="https://colab.research.google.com/github/vivshar/colab/blob/master/WordEmbeddingGlove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.preprocessing import sequence

Using TensorFlow backend.


In [0]:
import numpy as np

In [0]:
train_data = []
train_label = []

In [0]:
test_data =[]
test_label = []

In [0]:
import os

In [0]:
imdb_dir = '/content/aclImdb'
train_dir = os.path.join(imdb_dir,'train')
test_dir = os.path.join(imdb_dir,'test')

In [0]:
#train data
for label_name in ['neg','pos']:
  dir_name = os.path.join(train_dir,label_name)
  for file_name in os.listdir(dir_name):
    if file_name[-4:] == '.txt':
      f = open(os.path.join(dir_name,file_name))
      train_data.append(f.read())
      f.close()
      if label_name == 'neg':
        train_label.append(0)
      else:
        train_label.append(1)


In [0]:
from keras.preprocessing.text import Tokenizer
total_max_words = 10000

tokenizer = Tokenizer(num_words = total_max_words)

In [0]:
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index
train_seq = tokenizer.texts_to_sequences(train_data)

In [0]:
max_words = 100
train_seq = sequence.pad_sequences(train_seq,maxlen=max_words)
train_label = np.asarray(train_label)
indices = np.arange(train_seq.shape[0])
np.random.shuffle(indices)
train_seq = train_seq[indices]
train_label = train_label[indices]

In [0]:
training_samples = 200
train_sample = train_seq[:training_samples]
val_sample = train_seq[training_samples:]
train_sample_label = train_label[:training_samples]
val_sample_label = train_label[training_samples:]

In [0]:
embedding_index = {}
glove_dir = '/content/GloVe'
glove_output_dim = 300
f = open(os.path.join(glove_dir,'glove.6B.300d.txt'))
for line in f:
  values = line.split()
  word = values[0]
  embedding_index[word] = np.asarray(values[1:], dtype='float32')
f.close()

In [0]:
embedding_matrix = np.zeros((total_max_words,glove_output_dim))
for word,index in word_index.items():
  if index < total_max_words:
    coeff = embedding_index.get(word)
    if coeff is not None:
      embedding_matrix[index] = coeff

In [0]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,Embedding
model = Sequential()

In [0]:
model.add(Embedding(total_max_words,glove_output_dim,input_length = max_words))
model.add(Flatten())
model.add(Dense(32,activation= 'relu'))
model.add(Dense(1,activation='sigmoid'))

In [0]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          3000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                960032    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 3,960,065
Trainable params: 960,065
Non-trainable params: 3,000,000
_________________________________________________________________


In [0]:
model.fit(train_sample,train_sample_label,batch_size = 64, epochs = 10,validation_data = (val_sample,val_sample_label))

Train on 200 samples, validate on 24800 samples
Epoch 1/10
200/200 [==============================] - 3s 15ms/step - loss: 3.6330 - acc: 0.4950 - val_loss: 0.8473 - val_acc: 0.5006
Epoch 2/10
200/200 [==============================] - 3s 13ms/step - loss: 0.4420 - acc: 0.7200 - val_loss: 0.7173 - val_acc: 0.5639
Epoch 3/10
200/200 [==============================] - 3s 13ms/step - loss: 1.0006 - acc: 0.6550 - val_loss: 0.8628 - val_acc: 0.5065
Epoch 4/10
200/200 [==============================] - 3s 13ms/step - loss: 0.1531 - acc: 0.9600 - val_loss: 0.8194 - val_acc: 0.5466
Epoch 5/10
200/200 [==============================] - 3s 13ms/step - loss: 0.0572 - acc: 0.9950 - val_loss: 0.8789 - val_acc: 0.5419
Epoch 6/10
200/200 [==============================] - 3s 13ms/step - loss: 0.0294 - acc: 1.0000 - val_loss: 0.7998 - val_acc: 0.5648
Epoch 7/10
200/200 [==============================] - 3s 13ms/step - loss: 0.0176 - acc: 1.0000 - val_loss: 0.8454 - val_acc: 0.5606
Epoch 8/10
200/200 [=